In [ ]:
!wget -q https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import optimizers
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, RegexpTokenizer, TweetTokenizer, PorterStemmer


from re import sub
import tokenization
from tokenization import FullTokenizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
df_train= pd.read_csv('../input/amz-preprocessed-csv/amz_df.csv', escapechar = "\\", quoting = 3)
df_train['all']= (df_train.TITLE+ ' ' + df_train.DESCRIPTION+ ' ' + df_train.BULLET_POINTS+ ' ' + df_train.BRAND)
df_train= df_train[['all', 'BROWSE_NODE_ID']]
df_train= df_train.dropna()
df_train.head()

In [ ]:
#config
batch_size= 12
lr= 0.001
epoch= 50
patch= 80000

In [ ]:
df_mini= df_train[:2903024]
df_mini.BROWSE_NODE_ID= pd.to_numeric(df_mini.BROWSE_NODE_ID, errors='coerce')
del df_train

# df_test= pd.read_csv('../input/amz-data/dataset/test.csv', escapechar = "\\", quoting = 3)
# df= pd.concat([df_train, df_test])
# df.reset_index(inplace=True)
# print(df_train.shape, df_test.shape, df.shape)

In [ ]:
df_mini= df_mini.loc[df_mini['BROWSE_NODE_ID'] <= 2000]
df_mini.shape

In [ ]:
df_mini= df_mini[:patch]
df_mini= df_mini.dropna()
display(df_mini.head())
df_mini.shape

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
#tf.keras.utils.to_categorical(df_mini['BROWSE_NODE_ID'].values)
def generator_train():
    samples_per_epoch = X_train.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    while True:
        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0
        X= X_train[batch_size*counter:batch_size*(counter+1)]
        Y= y_train[batch_size*counter:batch_size*(counter+1)].astype('float32')
        counter += 1
        yield X,Y
        
        
def generator_test():
    samples_per_epoch = X_test.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    while True:
        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0
        X= X_test[batch_size*counter:batch_size*(counter+1)]
        Y= y_test[batch_size*counter:batch_size*(counter+1)].astype('float32')
        counter += 1
        yield X,Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_mini['all'][:patch].values,
            pd.get_dummies(df_mini.BROWSE_NODE_ID).values[:patch], test_size=0.2, random_state=22)
print(X_train.shape, X_test.shape)
train_steps= X_train.shape[0]//batch_size
val_steps= X_train.shape[0]//batch_size


train_iter= generator_train()
test_iter= generator_test()

# train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.string, tf.float32))
# train_iter = iter(train_ds)


# test_ds = tf.data.Dataset.from_generator(generator_test, output_types=(tf.string, tf.float32))
# test_iter = iter(test_ds)

In [ ]:
a, b= next(train_iter)
print(a.shape, b.shape)
df_mini.BROWSE_NODE_ID[:patch].unique().shape

In [ ]:
onehot2label= np.array(pd.get_dummies(df_mini.BROWSE_NODE_ID).columns)
np.save('1hot2label', onehot2label)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(1512, activation='relu'))
model.add(tf.keras.layers.Reshape((10, 128)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(512))
model.add(tf.keras.layers.Dropout(0.3))
model.add((tf.keras.layers.Dense(1046, activation='softmax')))
model.summary()

In [ ]:
name= 'AMZ_model_1lstm.h5'

rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)
        
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 6, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

In [ ]:
loss= tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizers.Adam(lr=lr),loss=loss,metrics=['acc'])

In [ ]:
history = model.fit_generator(train_iter, steps_per_epoch= train_steps, epochs=epoch,
                    validation_data= test_iter, validation_steps=val_steps, callbacks=[rlr, ckp, es])

<a href="./AMZ_model.h5" class="btn btn-primary">Start</a>

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.plot( history.history["loss"], label = "Training ACC" , marker='o')
plt.plot( history.history["val_loss"], label = "Validation AUC", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.plot( history.history["acc"], label = "Training ACC" , marker='o')
plt.plot( history.history["val_acc"], label = "Validation AUC", marker='+')
plt.grid(True)
plt.legend()
plt.show()